<a href="https://colab.research.google.com/github/harshvirMangla/DLbyHM/blob/main/TensorFlow_with_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tensorflow with GPU

This notebook provides an introduction to computing on a [GPU](https://cloud.google.com/gpu) in Colab. In this notebook you will connect to a GPU, and then run some basic TensorFlow operations on both the CPU and a GPU, observing the speedup provided by using the GPU.


## Enabling and testing the GPU

First, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

Next, we'll confirm that we can connect to the GPU with tensorflow:

In [1]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


## Observe TensorFlow speedup on GPU relative to CPU

This example constructs a typical convolutional neural network layer over a
random image and manually places the resulting ops on either the CPU or the GPU
to compare execution speed.

In [ ]:
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)

# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
3.862475891000031
GPU (s):
0.10837535100017703
GPU speedup over CPU: 35x


In [2]:
import tensorflow as tf
import numpy as np

def generate_data(num_samples=1000, sequence_length=10):
    x = np.random.rand(num_samples, sequence_length, 1)
    y = np.sum(x, axis=1)
    return x, y

x_data, y_data = generate_data()
train_size = int(0.8 * len(x_data))
x_train, y_train = x_data[:train_size], y_data[:train_size]
x_val, y_val = x_data[train_size:], y_data[train_size:]

model = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(32, activation='tanh', input_shape=(x_data.shape[1], x_data.shape[2])),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

history = model.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    epochs=8,
    batch_size=32
)

loss, mae = model.evaluate(x_val, y_val)
print(f"Validation Loss: {loss}, Validation MAE: {mae}")

test_input = np.random.rand(1, 10, 1)
predicted = model.predict(test_input)
print(f"Test input: {test_input.flatten()}, Predicted output: {predicted.flatten()}")

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/8
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 32ms/step - loss: 16.3760 - mae: 3.9018 - val_loss: 5.8269 - val_mae: 2.2285
Epoch 2/8
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 4.2502 - mae: 1.7928 - val_loss: 1.2549 - val_mae: 0.9279
Epoch 3/8
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.9499 - mae: 0.7861 - val_loss: 0.8740 - val_mae: 0.7562
Epoch 4/8
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.8597 - mae: 0.7459 - val_loss: 0.8636 - val_mae: 0.7507
Epoch 5/8
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.8511 - mae: 0.7416 - val_loss: 0.8415 - val_mae: 0.7475
Epoch 6/8
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.8056 - mae: 0.7223 - val_loss: 0.8197 - val_mae: 0.7395
Epoch 7/8
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.8376 - mae: 0.7435 - val_loss: 0.7932 - val_mae: 0.7250
Epoch 8/8
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.7404 - mae: 0.6892 - val_loss: 0.7629 - val_mae: 0.7174
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.7863 - mae: 0.7266 
Validation 

In [4]:
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.datasets import boston_housing

(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()
x_train = scaler_x.fit_transform(x_train)
x_test = scaler_x.transform(x_test)
y_train = scaler_y.fit_transform(y_train.reshape(-1, 1))
y_test = scaler_y.transform(y_test.reshape(-1, 1))

x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], 1)

model = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(32, activation='tanh', input_shape=(x_train.shape[1], x_train.shape[2])),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

history = model.fit(
    x_train, y_train,
    validation_data=(x_test, y_test),
    epochs=200,
    batch_size=32
)

loss, mae = model.evaluate(x_test, y_test)
print(f"Test Loss: {loss}, Test MAE: {mae}")

test_input = x_test[0:1]
predicted = model.predict(test_input)
print(f"Test input (scaled): {test_input.flatten()}, Predicted output (scaled): {predicted.flatten()}")

Epoch 1/200


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 71ms/step - loss: 0.3854 - mae: 0.5299 - val_loss: 0.1087 - val_mae: 0.2853
Epoch 2/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0834 - mae: 0.2354 - val_loss: 0.0410 - val_mae: 0.1600
Epoch 3/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0457 - mae: 0.1545 - val_loss: 0.0251 - val_mae: 0.1192
Epoch 4/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0293 - mae: 0.1248 - val_loss: 0.0209 - val_mae: 0.1083
Epoch 5/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0269 - mae: 0.1110 - val_loss: 0.0189 - val_mae: 0.1030
Epoch 6/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0205 - mae: 0.0995 - val_loss: 0.0168 - val_mae: 0.0972
Epoch 7/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0225 - mae: 0.1026 - val_loss: 0.0154 - val_mae: 0.0933
Epoch 8/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0191 - mae: 0.0940 - val_loss: 0.0145 - val_mae: 0.0898
Epoch 9/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0192 - mae: 0.

In [5]:

import tensorflow_datasets as tfds
raw_train_set, raw_valid_set, raw_test_set = tfds.load(
name="imdb_reviews",
split=["train[:90%]", "train[90%:]", "test"],
as_supervised=True
)
tf.random.set_seed(42)
train_set = raw_train_set.shuffle(5000, seed=42).batch(32).prefetch(1)
valid_set = raw_valid_set.batch(32).prefetch(1)
test_set = raw_test_set.batch(32).prefetch(1)

vocab_size = 10000

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.E06KS9_1.0.0/imdb_reviews-train.tfrecor…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.E06KS9_1.0.0/imdb_reviews-test.tfrecord…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.E06KS9_1.0.0/imdb_reviews-unsupervised.…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.
Epoch 1/2
704/704 ━━━━━━━━━━━━━━━━━━━━ 28s 34ms/step - accuracy: 0.4952 - loss: 0.6940 - val_accuracy: 0.5016 - val_loss: 0.6928
Epoch 2/2
704/704 ━━━━━━━━━━━━━━━━━━━━ 22s 32ms/step - accuracy: 0.5008 - loss: 0.6929 - val_accuracy: 0.5028 - val_loss: 0.6926


In [45]:

vocab_size = 30000
embed_size = 256
tf.random.set_seed(42)
model = tf.keras.Sequential([
text_vec_layer,
tf.keras.layers.Embedding(vocab_size, embed_size, mask_zero = True),
tf.keras.layers.GRU(256),
tf.keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="nadam",
metrics=["accuracy"])
history = model.fit(train_set, validation_data=valid_set, epochs=2)

Epoch 1/2
704/704 ━━━━━━━━━━━━━━━━━━━━ 76s 43ms/step - accuracy: 0.6404 - loss: 0.6130 - val_accuracy: 0.8576 - val_loss: 0.3298
Epoch 2/2
704/704 ━━━━━━━━━━━━━━━━━━━━ 39s 41ms/step - accuracy: 0.8638 - loss: 0.3221 - val_accuracy: 0.8696 - val_loss: 0.3076


In [58]:
comment = "Most parts were really good"

In [60]:
def process(com):
    return tf.convert_to_tensor([com])

comments = process(comment)
prediction = model.predict(comments)
print(prediction[0][0])

ValueError: Exception encountered when calling Sequential.call().

[1mCannot take the length of shape with unknown rank.[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=<unknown>, dtype=string)
  • training=False
  • mask=None

In [63]:
example_comment = tf.convert_to_tensor(["This is a test comment"])
try:
    prediction = model.predict(example_comment)
    print(f"Prediction: {prediction}")
except Exception as e:
    print(f"Error: {e}")

Error: Exception encountered when calling Sequential.call().

Cannot take the length of shape with unknown rank.

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=<unknown>, dtype=string)
  • training=False
  • mask=None


In [65]:
# Function to preprocess and predict a custom comment
def predict_custom_comment(comment):
    # Ensure comment is in batch format: wrap it in a list to make a batch of size 1
    processed_comment = tf.convert_to_tensor([comment])  # A batch of 1 comment

    # Predict sentiment
    prediction = model.predict(processed_comment)
    sentiment = "Positive" if prediction[0][0] > 0.5 else "Negative"

    return prediction[0][0], sentiment

# Example usage
custom_comment = "The movie was truly inspiring and beautifully made."
predicted_value, sentiment = predict_custom_comment(custom_comment)

print(f"Comment: {custom_comment}")
print(f"Prediction: {predicted_value:.2f}, Sentiment: {sentiment}")

ValueError: Exception encountered when calling Sequential.call().

[1mCannot take the length of shape with unknown rank.[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=<unknown>, dtype=string)
  • training=False
  • mask=None